In [1]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# python 버전 확인
!python --version

Python 3.8.13


In [3]:
# 판다스 버전 확인
pd.__version__

'1.4.3'

In [4]:
#!pip install selenium

In [4]:
# 검색어, 검색 기간 정의
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

1.크롤링할 키워드는 무엇입니까?: 홍대 맛집


In [5]:
query_txt

'홍대 맛집'

# 1. 크롤링할 블로그 url 수집하기

In [7]:
pwd

'/Users/aiden/Dropbox/Mac/Desktop'

In [5]:
options = webdriver.ChromeOptions()
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)

# 사이트 주소는 네이버
driver.get('https://search.shopping.naver.com/search/category/100002381?catId=50001092&frm=NVSHCHK&origQuery&pagingIndex=1&pagingSize=40&productSet=checkout&query&sort=rel&timestamp=&viewType=list')
time.sleep(2)  # 2초간 정지

In [7]:
# Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

In [8]:
# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )
time.sleep(1)

In [9]:
# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '옵션' 클릭
driver.find_element_by_link_text("옵션").click( )
time.sleep(1)

In [10]:
# 검색옵션 확인
item_li = driver.find_elements_by_css_selector('.option .txt')

for i in range(0, len(item_li)):
    print(item_li[i].text)

전체
블로그
카페
관련도순
최신순
전체
1시간
1일
1주
1개월
3개월
6개월
1년
직접입력옵션펼치기


In [11]:
# 검색기간 '3개월' 클릭
print(item_li[10].text)

item_li[10].click()

3개월


In [6]:
# 스크롤 다운
# driver.execute_script("window.scrollTo(0, 500)")
# time.sleep(2)

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 3
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1

In [7]:
# 블로그 글 url들 수집
url_list = []
title_list = []

# URL_raw 크롤링 시작
articles = ".basicList_link__1MaTN"
article_raw = driver.find_elements_by_css_selector(articles)
article_raw[0]

<selenium.webdriver.remote.webelement.WebElement (session="e55137b5e8953db27dd99240806e5849", element="8f5f64c9-cc4e-4570-9ae2-a28abd0bfc0f")>

In [8]:
# 첫번째 글의 url 확인
article_raw[0].get_attribute('href')

'https://adcr.naver.com/adcr?x=3tItLbCrm2Z+wf8xVADXj////w==kwlyIWaJhY9MhI6Vfnxb1vNcg30cOYp9U74hRzBd9dmFpyblYAwDkpNJhloyN9CUUEqGHn4hPDyTz1HYw9dmKlkSLbn0ME6sQ3DY/gp5nBVCw2hFyP5H/TbD6nou7mqzA8Ykc9R4oeAARJjU+5MqAlk2PTGtdhQWkQicdMoFp1OzdO5LLY5OXA3kKf81NLk9AkRQX29TQ/YDV+p3/Ys148DORmk53NS64gjKMxZs5JClRl3qL222+8zhOup5TdoFllkNZGTLg7CNTr4URtpK/N2H0JpyUjsYYq0GPc7UnAbI9V6mi4SpUjbN4JuYljawMteDcEupRxYPO7DjIGawAjTNASGlgCwalDrZvVEa63SpltuvGDElIhDuzs30pjh7JyDOmANbubTnqq7WdLTZku4VAkudODcgTfc0WyPcSGidPu9zHEPssiHSgbhSo5N8cE+VNULSZAvkSeBcAKQpDiFA17nj57aOgNiJL9iOZIan2ehmZPMwPge4jhMn9CQ1z+yZNcKKcS3zU7vvOlDoPLdXC/eaYpkkIhKBf3p9aA5ZL9K1XzfNvyHjtK9IS3AJxOb1OatxQ5vD8OAPzbd81oh/E8aSYXuAPvYUyXXoFYtkGGzjw5eN73njRDm74fJ3T4GLvIJB38uXdq6HV2PqxHsePasXoq7IUeVeZMF8kUbcDilcXN2bMwoq3xprVm5bpjd0xrCkCSo0Q1qiZt+n7MCZGYei+W/tuLz+URPC0XWO3zDD6YtwgXwHXcm8HNQWgEBk0Fhss6eD9ln4ZAXljoJEpEjM3Hdz0lSIdtaSPVvEyhssp4oEDSvvQmYTjq0rCyXxvNZuC+bkEngz4Ba6V0OlVS9GV3VEQxGIXtiMZbYICDZ2qTPVVlDkF2PJ4XSwz7KZ2DHc3S1wKf+A7MQliK1Yvcq6oqvRYakNuClTf+ri5

In [9]:
# 크롤링한 셀레늄 덩어리(객체)인 article_raw에서 get_attribute 함수를 이용해서 
# url 추출(href 속성값만 뽑아내기)하기
for article in article_raw:
    url = article.get_attribute('href')
    url_list.append(url)
time.sleep(1)

# 제목 크롤링 시작
for article in article_raw:
    title = article.text
    title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

[네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 식물성캡슐 차인표 루테인
[3+1] 5:1비율 뉴트리코어 루테인지아잔틴
뉴질랜드 라이프스팬 초록입홍합 오일추출 25000mg 120캡슐 푸른입홍합녹색그린
뉴트리원 정우성 루테인 지아잔틴 164 3박스 3개월분
닥터머콜라 퀘르세틴 케르세틴 프테로스틸벤 어드벤스 약들약 추천 60 180캡슐
정우성 유기농 밀크씨슬 2박스 2개월분 실리마린 간영양제
종근당건강 조인성 프로메가 알티지 오메가3듀얼 (3박스,3개월분)
연세 루테인 플러스 연세대학교 연세생활건강 4개월분
1+1 쏜리서치 SAT 60캡슐
뉴트리모어 액티브 엽산 400 800 임산부영양제 활성 활성형엽산 450mg x 60정 2개월
4개월분 4세대 활성형 활성엽산 리노브 액티폴릭 임산부 엽산 영양제
모유유래 bnr17 유산균 락토바실러스가세리 비에날17 다이어트 갱년기 영양제 yt1 비에날씬 여성 질 lgg 프로바이오틱스
뉴트리원 정우성 유산균 프로 프리 바이오틱스 3박스 3개월분
베이비락 이산화규소없는 유아 아기 유산균 아연함유 2개월분 신바이오틱스 신생아 어린이
유유제약 프리미엄 달맞이꽃종자유 감마리놀렌산 950mg 효능 효과 영양제 생리
종근당건강 조인성 프로메가 알티지 오메가3 듀얼 1박스
비타슈넬 식물성 캡슐 알티지 오메가3 2박스 2개월분
덴마크 유산균 이야기 12개월분 프로바이오틱스
피에이치365 킹 프로바이오틱스 베이비 아기유산균 신생아 모유 프롤린 영양제 돌
캐나다 더리얼 알티지 오메가3 맥스1400 초임계 식물성 장용성 임산부 epa dha
킹 프리미엄 생유산균 신바이오틱스포스트 모유 바이오틱스 프롤린lgg 키즈아기 여성질락토핏
트루포뮬러 비아노스 (전립선 건강의 유지에 도움)
닥터스베스트 MSM 식이유황 1500mg 120타블렛 옵티 Opti 엠에스엠 닥터베스트
버블레스 렐리시 여성 아연 질 유래 유산균 프로바이오틱스 30포(1개월분)
덴마크유산균이야기 4개월분 60캡슐 2박스 3세대 LGG유산균 시즌3
종근

In [10]:
# 수집된 url들 확인
url_list

['https://adcr.naver.com/adcr?x=3tItLbCrm2Z+wf8xVADXj////w==kwlyIWaJhY9MhI6Vfnxb1vNcg30cOYp9U74hRzBd9dmFpyblYAwDkpNJhloyN9CUUEqGHn4hPDyTz1HYw9dmKlkSLbn0ME6sQ3DY/gp5nBVCw2hFyP5H/TbD6nou7mqzA8Ykc9R4oeAARJjU+5MqAlk2PTGtdhQWkQicdMoFp1OzdO5LLY5OXA3kKf81NLk9AkRQX29TQ/YDV+p3/Ys148DORmk53NS64gjKMxZs5JClRl3qL222+8zhOup5TdoFllkNZGTLg7CNTr4URtpK/N2H0JpyUjsYYq0GPc7UnAbI9V6mi4SpUjbN4JuYljawMteDcEupRxYPO7DjIGawAjTNASGlgCwalDrZvVEa63SpltuvGDElIhDuzs30pjh7JyDOmANbubTnqq7WdLTZku4VAkudODcgTfc0WyPcSGidPu9zHEPssiHSgbhSo5N8cE+VNULSZAvkSeBcAKQpDiFA17nj57aOgNiJL9iOZIan2ehmZPMwPge4jhMn9CQ1z+yZNcKKcS3zU7vvOlDoPLdXC/eaYpkkIhKBf3p9aA5ZL9K1XzfNvyHjtK9IS3AJxOb1OatxQ5vD8OAPzbd81oh/E8aSYXuAPvYUyXXoFYtkGGzjw5eN73njRDm74fJ3T4GLvIJB38uXdq6HV2PqxHsePasXoq7IUeVeZMF8kUbcDilcXN2bMwoq3xprVm5bpjd0xrCkCSo0Q1qiZt+n7MCZGYei+W/tuLz+URPC0XWO3zDD6YtwgXwHXcm8HNQWgEBk0Fhss6eD9ln4ZAXljoJEpEjM3Hdz0lSIdtaSPVvEyhssp4oEDSvvQmYTjq0rCyXxvNZuC+bkEngz4Ba6V0OlVS9GV3VEQxGIXtiMZbYICDZ2qTPVVlDkF2PJ4XSwz7KZ2DHc3S1wKf+A7MQliK1Yvcq6oqvRYakNuClTf+ri

In [11]:
# 수집된 블로그 제목 확인
title_list

['[네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 식물성캡슐 차인표 루테인',
 '[3+1] 5:1비율 뉴트리코어 루테인지아잔틴',
 '뉴질랜드 라이프스팬 초록입홍합 오일추출 25000mg 120캡슐 푸른입홍합녹색그린',
 '뉴트리원 정우성 루테인 지아잔틴 164 3박스 3개월분',
 '닥터머콜라 퀘르세틴 케르세틴 프테로스틸벤 어드벤스 약들약 추천 60 180캡슐',
 '정우성 유기농 밀크씨슬 2박스 2개월분 실리마린 간영양제',
 '종근당건강 조인성 프로메가 알티지 오메가3듀얼 (3박스,3개월분)',
 '연세 루테인 플러스 연세대학교 연세생활건강 4개월분',
 '1+1 쏜리서치 SAT 60캡슐',
 '뉴트리모어 액티브 엽산 400 800 임산부영양제 활성 활성형엽산 450mg x 60정 2개월',
 '4개월분 4세대 활성형 활성엽산 리노브 액티폴릭 임산부 엽산 영양제',
 '모유유래 bnr17 유산균 락토바실러스가세리 비에날17 다이어트 갱년기 영양제 yt1 비에날씬 여성 질 lgg 프로바이오틱스',
 '뉴트리원 정우성 유산균 프로 프리 바이오틱스 3박스 3개월분',
 '베이비락 이산화규소없는 유아 아기 유산균 아연함유 2개월분 신바이오틱스 신생아 어린이',
 '유유제약 프리미엄 달맞이꽃종자유 감마리놀렌산 950mg 효능 효과 영양제 생리',
 '종근당건강 조인성 프로메가 알티지 오메가3 듀얼 1박스',
 '비타슈넬 식물성 캡슐 알티지 오메가3 2박스 2개월분',
 '덴마크 유산균 이야기 12개월분 프로바이오틱스',
 '피에이치365 킹 프로바이오틱스 베이비 아기유산균 신생아 모유 프롤린 영양제 돌',
 '캐나다 더리얼 알티지 오메가3 맥스1400 초임계 식물성 장용성 임산부 epa dha',
 '킹 프리미엄 생유산균 신바이오틱스포스트 모유 바이오틱스 프롤린lgg 키즈아기 여성질락토핏',
 '트루포뮬러 비아노스 (전립선 건강의 유지에 도움)',
 '닥터스베스트 MSM 식이유황 1500mg 120타블렛 옵티 Opti 엠에스엠

In [12]:
# 수집된 url_list, title_list로 판다스 데이터프레임 만들기
df = pd.DataFrame({'url':url_list, 'title':title_list})
df

,url,title
0,https://adcr.naver.com/adcr?x=3tItLbCrm2Z+wf8x...,[네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 ...
1,https://adcr.naver.com/adcr?x=++KJ6bPuBk25jS+V...,[3+1] 5:1비율 뉴트리코어 루테인지아잔틴
2,https://adcr.naver.com/adcr?x=MaspD5RFxKY+b5xP...,뉴질랜드 라이프스팬 초록입홍합 오일추출 25000mg 120캡슐 푸른입홍합녹색그린
3,https://cr.shopping.naver.com/adcr.nhn?x=9102z...,뉴트리원 정우성 루테인 지아잔틴 164 3박스 3개월분
4,https://cr.shopping.naver.com/adcr.nhn?x=Q%2BZ...,닥터머콜라 퀘르세틴 케르세틴 프테로스틸벤 어드벤스 약들약 추천 60 180캡슐
...,...,...
81,https://cr.shopping.naver.com/adcr.nhn?x=8%2FU...,우리아이유산균드롭스플러스D 닥터바이오D 신생아 돌 아기 유아 어린이 액상 유산균 비타민디
82,https://cr.shopping.naver.com/adcr.nhn?x=X1x6o...,뉴트리코어 브라질 그린 프로폴리스 NCS SIF인증
83,https://cr.shopping.naver.com/adcr.nhn?x=XV%2B...,미국 쏘팔메토 전립선영양제 옥타코사놀 소변자주마려움 1300mg
84,https://cr.shopping.naver.com/adcr.nhn?x=7Nhj2...,[정식수입] 바이오가이아 프로텍티스 이지 드롭 프로바이오틱스 +비타민D 10ml


In [13]:
pwd

'/Users/krc/TIL/RS/crawling'

In [14]:
# 저장하기
df.to_csv("blog_url.csv", encoding='utf-8-sig')
# df.to_csv("blog_url.csv", encoding='utf-8-sig')

# 2. 블로그 내용 크롤링하기

In [15]:
import sys
import os
import pandas as pd
import numpy as np

In [16]:
# "url_list.csv" 불러오기
url_load = pd.read_csv("blog_url.csv")
url_load = url_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

num_list = len(url_load)

print(num_list)
url_load

86


,url,title
0,https://adcr.naver.com/adcr?x=3tItLbCrm2Z+wf8x...,[네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 ...
1,https://adcr.naver.com/adcr?x=++KJ6bPuBk25jS+V...,[3+1] 5:1비율 뉴트리코어 루테인지아잔틴
2,https://adcr.naver.com/adcr?x=MaspD5RFxKY+b5xP...,뉴질랜드 라이프스팬 초록입홍합 오일추출 25000mg 120캡슐 푸른입홍합녹색그린
3,https://cr.shopping.naver.com/adcr.nhn?x=9102z...,뉴트리원 정우성 루테인 지아잔틴 164 3박스 3개월분
4,https://cr.shopping.naver.com/adcr.nhn?x=Q%2BZ...,닥터머콜라 퀘르세틴 케르세틴 프테로스틸벤 어드벤스 약들약 추천 60 180캡슐
...,...,...
81,https://cr.shopping.naver.com/adcr.nhn?x=8%2FU...,우리아이유산균드롭스플러스D 닥터바이오D 신생아 돌 아기 유아 어린이 액상 유산균 비타민디
82,https://cr.shopping.naver.com/adcr.nhn?x=X1x6o...,뉴트리코어 브라질 그린 프로폴리스 NCS SIF인증
83,https://cr.shopping.naver.com/adcr.nhn?x=XV%2B...,미국 쏘팔메토 전립선영양제 옥타코사놀 소변자주마려움 1300mg
84,https://cr.shopping.naver.com/adcr.nhn?x=7Nhj2...,[정식수입] 바이오가이아 프로텍티스 이지 드롭 프로바이오틱스 +비타민D 10ml


In [17]:
url_load['url']

0     https://adcr.naver.com/adcr?x=3tItLbCrm2Z+wf8x...
1     https://adcr.naver.com/adcr?x=++KJ6bPuBk25jS+V...
2     https://adcr.naver.com/adcr?x=MaspD5RFxKY+b5xP...
3     https://cr.shopping.naver.com/adcr.nhn?x=9102z...
4     https://cr.shopping.naver.com/adcr.nhn?x=Q%2BZ...
                            ...                        
81    https://cr.shopping.naver.com/adcr.nhn?x=8%2FU...
82    https://cr.shopping.naver.com/adcr.nhn?x=X1x6o...
83    https://cr.shopping.naver.com/adcr.nhn?x=XV%2B...
84    https://cr.shopping.naver.com/adcr.nhn?x=7Nhj2...
85    https://cr.shopping.naver.com/adcr.nhn?x=RfIAM...
Name: url, Length: 86, dtype: object

In [18]:
# 글 하나 띄우기
i = 1
url = url_load['url'][i]
driver = webdriver.Chrome(chrome_path, options=options)
driver.get(url)   # 글 띄우기

In [49]:
# iframe 접근
driver.switch_to.frame('mainFrame')

In [39]:
dict = {}
target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

In [34]:
# 제목 크롤링 시작
overlays = "._3oDjSvLwq9._copyable"                                 
tit = driver.find_element_by_css_selector(overlays)          # title
title = tit.text
title

'루테인 지아잔틴, 식물성 캡슐 뉴트리코어'

In [35]:
# 가격 크롤링 시작
overlays = "._1LY7DqCnwR"                                 
price = driver.find_element_by_css_selector(overlays)         # nickname
price = price.text
price

'190,000'

In [36]:
# matrix 크롤링
overlays = "._1_UiXWHt__"                                 
matrix = driver.find_element_by_css_selector(overlays)         # datetime
matrix = matrix.text
matrix

'상품번호 4683772291 상품상태 신상품\n브랜드 뉴트리코어 원산지 상세설명에 표시'

In [37]:
# matrix2 크롤링
overlays = ".detail_attributes ._1_UiXWHt__"                                 
matrix2 = driver.find_element_by_css_selector(overlays)         # datetime
matrix2 = matrix2.text
matrix2

'제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1정 제품용량 1개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민E, 베타카로틴, 비타민D, 엽산\n루테인 16mg'

In [40]:
# 글 하나는 target_info라는 딕셔너리에 담기게 되고,
target_info['title'] = title
target_info['price'] = price
target_info['matrix'] = matrix
target_info['matrix2'] = matrix2

target_info

{'title': '루테인 지아잔틴, 식물성 캡슐 뉴트리코어',
 'price': '190,000',
 'matrix': '상품번호 4683772291 상품상태 신상품\n브랜드 뉴트리코어 원산지 상세설명에 표시',
 'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1정 제품용량 1개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민E, 베타카로틴, 비타민D, 엽산\n루테인 16mg'}

In [41]:
dict[i] = target_info

dict

{1: {'title': '루테인 지아잔틴, 식물성 캡슐 뉴트리코어',
  'price': '190,000',
  'matrix': '상품번호 4683772291 상품상태 신상품\n브랜드 뉴트리코어 원산지 상세설명에 표시',
  'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1정 제품용량 1개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민E, 베타카로틴, 비타민D, 엽산\n루테인 16mg'}}

In [59]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

# 저장하기
result_df.to_excel("blog_content.xlsx", encoding='utf-8-sig')
time.sleep(3)

In [43]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

# 수집할 글 갯수 정하기
number = 10 

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome(chrome_path, options=options)
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성

        # 제목 크롤링 시작
        overlays = "._3oDjSvLwq9._copyable"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text
        title

        # 가격 크롤링 시작
        overlays = "._1LY7DqCnwR"                                 
        price = driver.find_element_by_css_selector(overlays)         # nickname
        price = price.text
        price

        # matrix 크롤링
        overlays = "._1_UiXWHt__"                                 
        matrix = driver.find_element_by_css_selector(overlays)         # datetime
        matrix = matrix.text
        matrix

        # matrix2 크롤링
        overlays = ".detail_attributes ._1_UiXWHt__"                                 
        matrix2 = driver.find_element_by_css_selector(overlays)         # datetime
        matrix2 = matrix2.text
        matrix2



        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['price'] = price
        target_info['matrix'] = matrix
        target_info['matrix2'] = matrix2

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
        
    # 판다스로 만들기
    import pandas as pd
    result_df = pd.DataFrame.from_dict(dict, 'index')

    # 저장하기
    result_df.to_csv("blog_content.csv", encoding='utf-8-sig')
    time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

  0%|          | 0/10 [00:00<?, ?it/s]

0 [네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 식물성캡슐 차인표 루테인
1 루테인 지아잔틴, 식물성 캡슐 뉴트리코어
2 뉴질랜드 라이프스팬 초록입홍합 오일추출 25000mg 120캡슐 푸른입홍합녹색그린
3 뉴트리원 정우성 루테인 지아잔틴 164 3박스 3개월분
4 닥터머콜라 퀘르세틴 케르세틴 프테로스틸벤 어드벤스 약들약 추천 60 180캡슐
5 정우성 유기농 밀크씨슬 2박스 2개월분 실리마린 간영양제
6 종근당건강 조인성 프로메가 알티지 오메가3듀얼 (3박스,3개월분)
7 연세 루테인 플러스 연세대학교 연세생활건강 4개월분
8 1+1 쏜리서치 SAT 60캡슐
9 뉴트리모어 액티브 엽산 400 800 임산부영양제 활성 활성형엽산 450mg x 60정 2개월
수집한 글 갯수:  10
{0: {'title': '[네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 식물성캡슐 차인표 루테인', 'price': '149,900', 'matrix': '상품번호 6606166860 상품상태 신상품\n브랜드 종근당건강 사은품 7/25~31일간 최대 7%적립!\n원산지 상세설명에 표시', 'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1캡슐 제품용량 10개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민A, 비타민D, 비타민E, 비타민B1, 비타민B6, 비타민B2\n루테인 18.2mg'}, 1: {'title': '루테인 지아잔틴, 식물성 캡슐 뉴트리코어', 'price': '190,000', 'matrix': '상품번호 4683772291 상품상태 신상품\n브랜드 뉴트리코어 원산지 상세설명에 표시', 'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1정 제품용량 1개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민E, 베타카로틴,

In [44]:
dict

{0: {'title': '[네이버 전용]종근당건강 아이클리어 루테인 지아잔틴 10박스(10개월분) 눈영양제 식물성캡슐 차인표 루테인',
  'price': '149,900',
  'matrix': '상품번호 6606166860 상품상태 신상품\n브랜드 종근당건강 사은품 7/25~31일간 최대 7%적립!\n원산지 상세설명에 표시',
  'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1캡슐 제품용량 10개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민A, 비타민D, 비타민E, 비타민B1, 비타민B6, 비타민B2\n루테인 18.2mg'},
 1: {'title': '루테인 지아잔틴, 식물성 캡슐 뉴트리코어',
  'price': '190,000',
  'matrix': '상품번호 4683772291 상품상태 신상품\n브랜드 뉴트리코어 원산지 상세설명에 표시',
  'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1정 제품용량 1개월분\n주요 기능성(식약처인증) 눈건강 영양소 원료명(식약처고시) 비타민E, 베타카로틴, 비타민D, 엽산\n루테인 16mg'},
 2: {'title': '뉴질랜드 라이프스팬 초록입홍합 오일추출 25000mg 120캡슐 푸른입홍합녹색그린',
  'price': '238,000',
  'matrix': '상품번호 3472701150 상품상태 신상품\n제조사 라이프스팬 브랜드 라이프스팬\n모델명 라이프스팬 울트라플렉스 초록입홍합오일 25000mg 120캡슐 사은품 구매가격 10만원 이상시 액수따라 비누,치약,크림 등 사은품 증정\n원산지 뉴질랜드산(뉴질랜드)',
  'matrix2': '제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1정 제품용량 4개월분'},
 3: {'title': '뉴트리원 정우성 루테인 지아잔틴 164 3박스 3개월분

In [40]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

,title,nickname,datetime,content
1,합정 홍대 라멘맛집 오레노라멘 미쉐린2022 선정 (주차팁),잇츠마인\n(love_od),2022. 6. 18. 16:00,홍대 라멘맛집이라고 검색이 더 많이되고있지만\n실제로는 합정역에서 더 가까운 오레노...
2,홍대 합정 맛집 포36거리 월남쌈 쌀국수 세트 추천!,먹슬지,2022. 7. 14. 0:06,먹슬지입니다~~👋\n오늘은 월남쌈이 먹고 싶어서 찾아갔던\n홍대 합정 맛집 소개해...
3,홍대 씨부엉 통문어가 들어간 중화풍 즉석떡볶이 맛집,바카,2022. 7. 8. 7:00,홍대 미대 나온 염님을 보필하고 사는 노예 바카님이라\n가끔씩 염님 추억팔이하러 홍...
4,홍대 고기집 합정 맛집 고기꾼김춘배 친절하고 맛있음,미니미엄빠,2022. 6. 30. 11:16,#홍대고기집 #합정맛집\n고기꾼김춘배 홍대본점 홍대 거리에서 너무나도 유명한 고기집...
5,ㅣ홍대맛집ㅣ서교동맛집 '스테이크 슈퍼' (+어린이메뉴),몽쉘미쉘\n(xxchocoxx),2022. 7. 8. 10:41,스테이크 슈퍼 http://instagram.com/steaksuper_offici...
6,"[홍대 맛집] 홍대의 일본식 야끼카레 맛집, 비스트로 큐슈",템포,2022. 6. 19. 8:50,다양한 맛집이 많이 있는 홍대\n매번 홍대에 갈 때마다\n너무나도 많은 맛집들이 있...
7,홍대 맛집 '프리모바치오바치' 빠네 파스타 맛집 (+꿀팁),싱깅\n(tldrld93),2022. 5. 13. 5:30,"홍대 맛집, 홍대 파스타\n프리모바치오바치 저번 주 금요일, 쟁의 청첩장 모임이 있..."
8,[홍대맛집/키움초밥]맛과 가성비 다 잡은캐쥬얼초밥집.데이트 하고 식사하러 가셔도 좋을듯.,바블,2022. 7. 9. 7:00,아침부터 홍대에서 시작한\n업무가 점심쯔음 서교동에서 마쳤어요\n일 땜시 여동생을 ...
9,홍대입구텐동맛집 [고구텐 홍대점] 항상 사람들로 가득한 홍대 맛집,먹는데 진심인 오사장\n(ops1200),2022. 7. 1. 13:59,매장번호 0507-1331-5610\n영업시간\n11:30 - 21:00\n15:0...


In [41]:
# 엑셀로 저장하기
result_df.to_csv("blog_content.csv", encoding='utf-8-sig')

In [26]:
pwd

'/Users/aiden/Dropbox/Mac/Desktop/크롤러'